In [ ]:
from autoencoder import Autoencoder
from parser import data_converter, print_letter
import numpy as np
import matplotlib.pyplot as plt
from compare_architectures import sample_set

In [ ]:
testing_set = list()
training_set = None
for i in [1]:
        labels, alphabet = data_converter("resources/fonts_" + str(i) + ".txt")
        alphabet = np.array(alphabet)
        flattened_input = np.array(list(map(lambda char: np.array(char).flatten(), alphabet)))
        if training_set is None:
                training_set = flattened_input
        else:
                training_set = np.concatenate(training_set, sample_set(flattened_input, 0.5))
        testing_set.append(list(zip(labels, flattened_input)))

# for char in alphabet:
#     print_letter(char)
# print(len(alphabet))


In [ ]:
epochs = []
errors =[]

def callback_fun(e, error, w, o):
    epochs.append(e)
    errors.append(error)

In [ ]:

activation_function = (lambda x: 1/(1+np.exp(-2*x)))
activation_derivative = (lambda x: 2*(1/(1+np.exp(-2*x)))*(1-(1/(1+np.exp(-2*x)))) )
ae = Autoencoder(35, [], latent_dim=2, activation_function=activation_function,activation_function_derivative=activation_derivative, update_learn_rate=None, learning_rate=0.001, update_frequency=0,momentum=False, use_adam=False)


output, err_min = ae.train(training_set, expected_output=training_set, epoch_limit=100, callback=callback_fun)

test_error = ae.test_autoencoder(testing_set[0])
print(test_error)
plt.plot(epochs, errors)

In [ ]:
epochs = []
errors =[]
ae = Autoencoder(35, [], latent_dim=2, activation_function=activation_function,activation_function_derivative=activation_derivative, update_learn_rate=None, learning_rate=0.001, update_frequency=len(training_set),momentum=False, use_adam=False)


output, err_min = ae.train(training_set, expected_output=training_set, epoch_limit=100, callback=callback_fun)

test_error = ae.test_autoencoder(testing_set[0])
print(test_error)
plt.plot(epochs, errors)

In [ ]:
epochs = []
errors =[]
ae = Autoencoder(35, [], latent_dim=2, activation_function=activation_function,activation_function_derivative=activation_derivative, update_learn_rate=None, learning_rate=0.001, update_frequency=len(training_set),momentum=True, use_adam=False)


output, err_min = ae.train(training_set, expected_output=training_set, epoch_limit=100, callback=callback_fun)

test_error = ae.test_autoencoder(testing_set[0])
print(test_error)
plt.plot(epochs, errors)

In [ ]:
epochs = []
errors =[]
ae = Autoencoder(35, [], latent_dim=2, activation_function=activation_function,activation_function_derivative=activation_derivative, update_learn_rate=None, learning_rate=0.001, update_frequency=0,momentum=True, use_adam=False)


output, err_min = ae.train(training_set, expected_output=training_set, epoch_limit=100, callback=callback_fun)

test_error = ae.test_autoencoder(testing_set[0])
print(test_error)
plt.plot(epochs, errors)

In [ ]:
epochs = []
errors =[]
def adaptive_eta(eta, error_k):
    consistent = 3 # La cantidad de epocas a partir de las cuales considero que el error crece o decrese _consistentemente_ 
    a = 0.01 # Constante de ajuste si el error crece
    b = 0.03 #Ajuste si el error decrece 
    if error_k > consistent and error_k % consistent == 0: 
        #print("Error was increasing, adapt eta. Error_k: ", error_k) 
      return eta - b*eta 
        
        
        
    elif error_k < -consistent and error_k % consistent == 0:
        #print("Error was decreasing, adapt eta, Error_k: ", error_k) 
        return eta + a

    else:
        return eta

def no_adaptive_eta(eta, error_k):
    return eta

ae = Autoencoder(35, [25, 17], latent_dim=2, activation_function=activation_function,activation_function_derivative=activation_derivative, update_learn_rate=adaptive_eta, learning_rate=0.001, update_frequency=0,momentum=False, use_adam=False)


output, err_min = ae.train(training_set, expected_output=training_set, epoch_limit=100, callback=callback_fun)

test_error = ae.test_autoencoder(testing_set[0])
print(test_error)
plt.plot(epochs, errors)

In [ ]:
# i = 10
# print_letter(alphabet[i])
# reconstruct, reconstruction_error = ae.reconstruct(flattened_input[i])

# print_letter(reconstruct.reshape(alphabet[i].shape))
# print(reconstruction_error)

# latent_output = ae.encode(flattened_input[i])
# print(latent_output)
# print_letter(ae.decode(latent_output).reshape(alphabet[i].shape))

# ae.graph_latent_space(flattened_input)



In [ ]:
from compare_architectures import compare_architectures
#compare_architectures()

In [ ]:
from parser import parse_config
from compare_architectures import graph_evolution, graph_latent_space, graph_latent_space_by_font, graph_comp_error
config_file = parse_config("resources/config.json")
input_values = list(map(lambda a: str(a), config_file["architectures"]))
graph_evolution(0, input_values, comparing_attribute="architecture")


In [ ]:
graph_latent_space(0, input_values, comparing_attribute="architecture", which_font=-1, show_labels=False)


In [ ]:
graph_latent_space(0, ["[25, 17, 8, 4]", "[17, 8, 4]" ], comparing_attribute="special_architecture", which_font=-1, show_labels=True)

Same architecture, divide by font:

In [ ]:
graph_latent_space_by_font(0, ["[25, 17]"], comparing_attribute="special_architecture", show_labels=True, which_fonts=[0,1,2])

Two best: 

In [ ]:
graph_latent_space(0, ["[25, 17]", "[25, 17, 8]" ], comparing_attribute="special_architecture", which_font=0, show_labels=True)

Two with worse error: 

In [ ]:
graph_latent_space(0, ["[25, 17, 8, 4]", "[17, 8, 4]" ], comparing_attribute="special_architecture", which_font=0, show_labels=True)

In [ ]:
graph_comp_error(input_values, comparing_attribute="architecture", execution_count=2, separate_by_font=False)

In [ ]:
n = 5  # figure with 15x15 digits
digit_size = 80
figure = np.zeros((digit_size * n, digit_size * n))
# linearly spaced coordinates on the unit square were transformed through the inverse CDF (ppf) of the Gaussian
# to produce values of the latent variables z, since the prior of the latent space is Gaussian
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))
print(grid_y)
for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        x_decoded = decoder.predict(z_sample)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
               j * digit_size: (j + 1) * digit_size] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='Greys_r')
plt.show()